# Example 9: Activations in Two Modes

This notebook demonstrates how to:
1. Load Bielik model
2. Mode 1: Save activations from texts using `save_activations()`
3. Mode 2: Save activations from dataset using `save_activations_dataset()`
4. Verify activations were saved correctly in both modes

This shows the activations API that automatically attaches detectors and saves activations.

In [1]:
# Setup and imports
%load_ext autoreload
%autoreload 2

import torch
from pathlib import Path
from datetime import datetime

from amber.datasets import TextDataset
from amber.language_model.language_model import LanguageModel
from amber.store.local_store import LocalStore
from datasets import load_dataset

print("✅ Imports completed")

/Users/adam/Projects/Inzynierka/codebase/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports completed


In [2]:
# Configuration
MODEL_ID = "speakleash/Bielik-1.5B-v3.0-Instruct"
STORE_DIR = Path("store")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
MAX_LENGTH = 128
DATA_LIMIT = 10

HF_DATASET = "roneneldan/TinyStories"
TEXT_FIELD = "text"
DATA_SPLIT = "train"

print("⚙️ Configuration:")
print(f"   Model: {MODEL_ID}")
print(f"   Device: {DEVICE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Max length: {MAX_LENGTH}")
print(f"   Dataset: {HF_DATASET}")
print(f"   Data limit: {DATA_LIMIT} samples")

⚙️ Configuration:
   Model: speakleash/Bielik-1.5B-v3.0-Instruct
   Device: cpu
   Batch size: 4
   Max length: 128
   Dataset: roneneldan/TinyStories
   Data limit: 10 samples


In [3]:
# Step 1: Load Bielik model
print("📥 Loading Bielik model...")

store = LocalStore(STORE_DIR)
lm = LanguageModel.from_huggingface(MODEL_ID, store=store)
lm.model.to(DEVICE)

print(f"✅ Model loaded: {lm.model_id}")
print(f"📱 Device: {DEVICE}")
print(f"📁 Store location: {lm.store.base_path}")

📥 Loading Bielik model...
✅ Model loaded: speakleash_Bielik-1.5B-v3.0-Instruct
📱 Device: cpu
📁 Store location: store


In [4]:
# Step 2: Find a layer to capture activations from
print("🔍 Finding layer to attach activation detector...")

layer_names = lm.layers.get_layer_names()
print(f"📋 Found {len(layer_names)} layers")

transformer_layers = [name for name in layer_names if 'transformer' in name.lower() or 'layer' in name.lower()]
if transformer_layers:
    attention_layers = [name for name in transformer_layers if 'attn' in name.lower()]
    if attention_layers:
        LAYER_SIGNATURE = attention_layers[0]
    else:
        LAYER_SIGNATURE = transformer_layers[0]
else:
    LAYER_SIGNATURE = layer_names[0] if layer_names else None

if LAYER_SIGNATURE:
    print(f"✅ Selected layer: {LAYER_SIGNATURE}")
else:
    raise ValueError("Could not find a suitable layer")

🔍 Finding layer to attach activation detector...
📋 Found 422 layers
✅ Selected layer: llamaforcausallm_model_layers_0_self_attn


In [5]:
# Step 3: Create small dataset
print("📊 Creating dataset...")

hf_dataset = load_dataset(HF_DATASET, split=DATA_SPLIT, streaming=False)
if DATA_LIMIT > 0:
    hf_dataset = hf_dataset.select(range(min(DATA_LIMIT, len(hf_dataset))))

dataset = TextDataset(hf_dataset, store=store, text_field=TEXT_FIELD)

print(f"✅ Dataset created: {len(dataset)} samples")
print(f"📝 Sample text: {dataset[0][:100]}..." if len(dataset[0]) > 100 else f"📝 Sample text: {dataset[0]}")

📊 Creating dataset...


Saving the dataset (1/1 shards): 100%|██████████| 10/10 [00:00<00:00, 5098.84 examples/s]

✅ Dataset created: 10 samples
📝 Sample text: One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with...


In [6]:
# Step 4: Mode 1 - Save activations from texts using save_activations()
print("🚀 Mode 1: Save activations from texts using save_activations()")
print("=" * 60)
print()

texts = [dataset[i] for i in range(min(6, len(dataset)))]
print(f"📝 Processing {len(texts)} texts...")
print(f"   Layer: {LAYER_SIGNATURE}")
print()

run_name_texts = f"activations_texts_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
print(f"📁 Run name: {run_name_texts}")
print()

run_name = lm.activations.save_activations(
    texts,
    layer_signature=LAYER_SIGNATURE,
    run_name=run_name_texts,
    batch_size=3,
    max_length=MAX_LENGTH,
    autocast=False,
    verbose=True,
)

print()
print(f"✅ Activations saved!")
print(f"📁 Run name: {run_name}")
print()

batches = lm.store.list_run_batches(run_name)
print(f"📦 Saved {len(batches)} batches to store")

2025-12-09 22:07:26,763 [INFO] amber.language_model.activations: Starting save_activations: run=activations_texts_20251209_220726, layer=llamaforcausallm_model_layers_0_self_attn, batch_size=3, device=cpu


🚀 Mode 1: Save activations from texts using save_activations()

📝 Processing 6 texts...
   Layer: llamaforcausallm_model_layers_0_self_attn

📁 Run name: activations_texts_20251209_220726



2025-12-09 22:07:28,884 [INFO] amber.language_model.activations: Saved batch 0 for run=activations_texts_20251209_220726
2025-12-09 22:07:29,674 [INFO] amber.language_model.activations: Saved batch 1 for run=activations_texts_20251209_220726
2025-12-09 22:07:29,674 [INFO] amber.language_model.activations: Completed save_activations: run=activations_texts_20251209_220726, batches_saved=2



✅ Activations saved!
📁 Run name: activations_texts_20251209_220726

📦 Saved 2 batches to store


In [7]:
# Step 5: Verify saved activations from save_activations()
print("🔍 Verifying saved activations from save_activations()...")
print()

if len(batches) > 0:
    batch_idx = 0
    retrieved_metadata, retrieved_tensors = lm.store.get_detector_metadata(run_name, batch_idx)
    
    print(f"📊 Batch {batch_idx} structure:")
    print(f"   Layers with data: {list(retrieved_tensors.keys())}")
    print()
    
    if str(LAYER_SIGNATURE) in retrieved_tensors:
        activations = retrieved_tensors[str(LAYER_SIGNATURE)].get("activations")
        if activations is not None:
            print(f"✅ Activations found:")
            print(f"   Shape: {activations.shape}")
            print(f"   Dtype: {activations.dtype}")
            print(f"   Device: {activations.device}")
        else:
            print("❌ Activations not found")
    else:
        print(f"❌ Layer {LAYER_SIGNATURE} not found in saved data")
    
    run_metadata = lm.store.get_run_metadata(run_name)
    if run_metadata:
        print()
        print(f"✅ Run metadata found:")
        print(f"   Model: {run_metadata.get('model', 'N/A')}")
        print(f"   Layer signatures: {run_metadata.get('layer_signatures', 'N/A')}")
        print(f"   Batch size: {run_metadata.get('options', {}).get('batch_size', 'N/A')}")
else:
    print("❌ No batches found")

🔍 Verifying saved activations from save_activations()...

📊 Batch 0 structure:
   Layers with data: ['llamaforcausallm_model_layers_0_self_attn']

✅ Activations found:
   Shape: torch.Size([3, 128, 1536])
   Dtype: torch.float32
   Device: cpu

✅ Run metadata found:
   Model: LlamaForCausalLM
   Layer signatures: ['llamaforcausallm_model_layers_0_self_attn']
   Batch size: 3


In [8]:
# Step 6: Mode 2 - Save activations from dataset using save_activations_dataset()
print("🚀 Mode 2: Save activations from dataset using save_activations_dataset()")
print("=" * 60)
print()

run_name_dataset = f"activations_dataset_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
print(f"📁 Run name: {run_name_dataset}")
print(f"📊 Dataset size: {len(dataset)} samples")
print(f"   Layer: {LAYER_SIGNATURE}")
print(f"📦 Batch size: {BATCH_SIZE}")
print()

run_name = lm.activations.save_activations_dataset(
    dataset,
    layer_signature=LAYER_SIGNATURE,
    run_name=run_name_dataset,
    batch_size=BATCH_SIZE,
    max_length=MAX_LENGTH,
    autocast=False,
    verbose=True,
)

print()
print(f"✅ Activations saved!")
print(f"📁 Run name: {run_name}")
print()

batches = lm.store.list_run_batches(run_name)
print(f"📦 Saved {len(batches)} batches to store")

2025-12-09 22:07:29,724 [INFO] amber.language_model.activations: Starting save_activations_dataset: run=activations_dataset_20251209_220729, layer=llamaforcausallm_model_layers_0_self_attn, batch_size=4, device=cpu


🚀 Mode 2: Save activations from dataset using save_activations_dataset()

📁 Run name: activations_dataset_20251209_220729
📊 Dataset size: 10 samples
   Layer: llamaforcausallm_model_layers_0_self_attn
📦 Batch size: 4



2025-12-09 22:07:30,775 [INFO] amber.language_model.activations: Saved batch 0 for run=activations_dataset_20251209_220729
2025-12-09 22:07:31,832 [INFO] amber.language_model.activations: Saved batch 1 for run=activations_dataset_20251209_220729
2025-12-09 22:07:32,405 [INFO] amber.language_model.activations: Saved batch 2 for run=activations_dataset_20251209_220729
2025-12-09 22:07:32,405 [INFO] amber.language_model.activations: Completed save_activations_dataset: run=activations_dataset_20251209_220729, batches_saved=3



✅ Activations saved!
📁 Run name: activations_dataset_20251209_220729

📦 Saved 3 batches to store


In [9]:
# Step 7: Verify saved activations from save_activations_dataset()
print("🔍 Verifying saved activations from save_activations_dataset()...")
print()

if len(batches) > 0:
    batch_idx = 0
    retrieved_metadata, retrieved_tensors = lm.store.get_detector_metadata(run_name, batch_idx)
    
    print(f"📊 Batch {batch_idx} structure:")
    print(f"   Layers with data: {list(retrieved_tensors.keys())}")
    print()
    
    if str(LAYER_SIGNATURE) in retrieved_tensors:
        activations = retrieved_tensors[str(LAYER_SIGNATURE)].get("activations")
        if activations is not None:
            print(f"✅ Activations found:")
            print(f"   Shape: {activations.shape}")
            print(f"   Dtype: {activations.dtype}")
            print(f"   Device: {activations.device}")
        else:
            print("❌ Activations not found")
    else:
        print(f"❌ Layer {LAYER_SIGNATURE} not found in saved data")
    
    run_metadata = lm.store.get_run_metadata(run_name)
    if run_metadata:
        print()
        print(f"✅ Run metadata found:")
        print(f"   Model: {run_metadata.get('model', 'N/A')}")
        print(f"   Layer signatures: {run_metadata.get('layer_signatures', 'N/A')}")
        print(f"   Batch size: {run_metadata.get('options', {}).get('batch_size', 'N/A')}")
        print(f"   Dataset length: {run_metadata.get('dataset', {}).get('length', 'N/A')}")
    
    print()
    print(f"📊 All batches summary:")
    for i in range(min(3, len(batches))):
        meta, tensors = lm.store.get_detector_metadata(run_name, i)
        acts = tensors.get(str(LAYER_SIGNATURE), {}).get("activations", None)
        if acts is not None:
            print(f"   Batch {i}: activations shape {acts.shape}")
else:
    print("❌ No batches found")

🔍 Verifying saved activations from save_activations_dataset()...

📊 Batch 0 structure:
   Layers with data: ['llamaforcausallm_model_layers_0_self_attn']

✅ Activations found:
   Shape: torch.Size([4, 128, 1536])
   Dtype: torch.float32
   Device: cpu

✅ Run metadata found:
   Model: LlamaForCausalLM
   Layer signatures: ['llamaforcausallm_model_layers_0_self_attn']
   Batch size: 4
   Dataset length: 10

📊 All batches summary:
   Batch 0: activations shape torch.Size([4, 128, 1536])
   Batch 1: activations shape torch.Size([4, 128, 1536])
   Batch 2: activations shape torch.Size([2, 128, 1536])


In [10]:
# Step 8: Compare both modes
print("📊 Comparison of both modes")
print("=" * 60)
print()

print("Mode 1: save_activations() - for list of texts")
print(f"   Run name: {run_name_texts}")
batches_texts = lm.store.list_run_batches(run_name_texts)
print(f"   Batches saved: {len(batches_texts)}")
if len(batches_texts) > 0:
    meta_texts, tensors_texts = lm.store.get_detector_metadata(run_name_texts, 0)
    acts_texts = tensors_texts.get(str(LAYER_SIGNATURE), {}).get("activations", None)
    if acts_texts is not None:
        print(f"   Activations shape (batch 0): {acts_texts.shape}")
print()

print("Mode 2: save_activations_dataset() - for dataset")
print(f"   Run name: {run_name_dataset}")
batches_dataset = lm.store.list_run_batches(run_name_dataset)
print(f"   Batches saved: {len(batches_dataset)}")
if len(batches_dataset) > 0:
    meta_dataset, tensors_dataset = lm.store.get_detector_metadata(run_name_dataset, 0)
    acts_dataset = tensors_dataset.get(str(LAYER_SIGNATURE), {}).get("activations", None)
    if acts_dataset is not None:
        print(f"   Activations shape (batch 0): {acts_dataset.shape}")
print()

print("✅ Both modes work correctly and save activations in the same format!")

📊 Comparison of both modes

Mode 1: save_activations() - for list of texts
   Run name: activations_texts_20251209_220726
   Batches saved: 2
   Activations shape (batch 0): torch.Size([3, 128, 1536])

Mode 2: save_activations_dataset() - for dataset
   Run name: activations_dataset_20251209_220729
   Batches saved: 3
   Activations shape (batch 0): torch.Size([4, 128, 1536])

✅ Both modes work correctly and save activations in the same format!


## Summary

This example demonstrated:

1. ✅ **Loading Bielik model** - Successfully loaded from HuggingFace
2. ✅ **Mode 1: save_activations()** - Save activations from list of texts
3. ✅ **Mode 2: save_activations_dataset()** - Save activations from dataset
4. ✅ **Verification** - Confirmed activations saved correctly in both modes

**Key Benefits:**
- `save_activations()` - Simple activation saving for text lists with automatic detector management
- `save_activations_dataset()` - Efficient batch processing of datasets with activation saving
- Both methods automatically attach and cleanup detectors
- Both methods save metadata consistently
- Activations are saved in the same format regardless of input source

**Conclusion:** ✅ The activations API provides convenient methods for saving activations from both texts and datasets!